In [1]:
from openai import OpenAI
import json
import csv
from tqdm import tqdm
with open('/burg/home/nd2794/keys.json', 'r') as file:
    keys = json.load(file)

In [2]:
client = OpenAI(api_key=keys['openai_token'])

In [3]:
system_message = {
    "role": "system",
    "content": [
        {
            "text": "In this task, you will be given two texts which summarize two different events. \nThe two events may be from the same context, but may also be unrelated or have a weak relationship.\nYour task is to determine if the two events have a likely temporal relation, meaning one procedes the other.\n\nEach problem has all of the following information:\n- A summary of the two events, each in a different line",
            "type": "text"
        }
    ]
}

user_message_1 = {
    "role": "user",
    "content": [
        {
            "text": "Event 1: The man in the grey and white shirt enters the enclosed squash court picks up some of the balls and proceeds to load the squash cannon serving machines.\nEvent 2: A man wearing a white and grey shirt serves in a practice squash session and another man wearing a purple shirt returns the serves in an enclosed squash court.\n\nDo the two events summarized above have a temporal relationship? Please choose between Yes and No. Please answer in json format with explanation.",
            "type": "text"
        }
    ]
}

assistant_messge_1 = {
    "role": "assistant",
    "content": [
        {
            "text": "{\"answer\": \"Yes\", \"explanation\": \"The serving machine must be loaded with balls before the man can practice serving.\"}",
            "type": "text"
        }
    ]
}

user_message_2 = {
    "role": "user",
    "content": [
        {
            "text": "Event 1: Sumo wrestlers lift up legs and then crouch .\nEvent 2: Sumo wrestlers eat food in the dojo .\n\nDo the two events summarized above have a temporal relationship? Please choose between Yes and No. Please answer in json format with explanation.",
            "type": "text"
        }
    ]
}

assistant_messge_2 = {
    "role": "assistant",
    "content": [
        {
            "text": "{\"answer\": \"No\", \"explanation\": \"It is ambiguous which event comes first as neither must necessarily precede the other.\"}",
            "type": "text"
        }
    ]
}

INSTRUCTION = "Do the two events summarized above have a temporal relationship? Please choose between Yes and No. Please answer in json format with explanation."

In [4]:
def run_llama_subset():
    fname = '../data_viz/samples_200.csv'
    with open(fname, newline='') as f:
        reader = csv.reader(f)
        data = list(reader)

    llama_outputs = []

    for sample in tqdm(data[1:]):
        idx = sample[0]
        um_3 = 'Event 1: ' + sample[1] + '\nEvent 2: ' + sample[2] + '\n\n' + INSTRUCTION

        user_message_3 = {
            "role": "user",
            "content": [
                {
                    "text": um_3,
                    "type": "text"
                }
            ]
        }

        messages=[
            system_message,
            user_message_1,
            assistant_messge_1,
            user_message_2,
            assistant_messge_2,
            user_message_3
        ]

        response = client.chat.completions.create(
            model="gpt-4o-2024-05-13",
            response_format={"type": "json_object"},
            messages=messages,
            temperature=0.5,
            max_tokens=3000,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0
        )

        answer_json_str1 = response.choices[0].message.content
        
        try:
            answer_json = json.loads(answer_json_str1)
            answer = answer_json['answer']
            explanation = answer_json['explanation']
        except:
            try:
                last_inst_idx = output.rfind('{')
                answer_json_str2 = output[last_inst_idx:]
                answer_json = json.loads(answer_json_str2)
                answer = answer_json['answer']
                explanation = answer_json['explanation']
            except:
                answer = 'unclear'
                explanation = 'none'

        sample[-1] = answer
        sample.append(explanation)

    with open('../data_viz/samples_200_openai-gpt-4o-2024-05-13_temp_0.5.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerows(data)
    return

In [6]:
run_llama_subset()

  0%|          | 0/200 [00:02<?, ?it/s]


APIConnectionError: Connection error.